In [ ]:
# Install dependencies for ALLaM and Bohour
!pip install transformers datasets torch accelerate bitsandbytes wandb arabic-reshaper python-bidi
!pip install git+https://github.com/MagedSaeed/Bohour.git
!pip install -U transformers sentencepiece accelerate datasets evaluate
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 33.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
import os
from google.colab import drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define paths
save_path = "/content/drive/MyDrive/allam_fine_tuned_Ashar_v2"
checkpoint_path = os.path.join(save_path, "checkpoint-6894")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)

# Load the base ALLaM model
base_model_name = "ALLaM-AI/ALLaM-7B-Instruct-preview"
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    load_in_8bit=True  # Matches your training setup
)

# Load the PEFT (LoRA) checkpoint
model = PeftModel.from_pretrained(base_model, checkpoint_path)
model.eval()  # Set to evaluation mode

# Ensure padding token consistency
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

print("Model and tokenizer loaded from Checkpoint-6000")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.03G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Model and tokenizer loaded from Checkpoint-6000


In [ ]:
# Helper to build the input prompt
ST_BAYT_TOKEN = "<|bayt|>"
VERSE_TOKEN   = "<|verse|>"
ED_BAYT_TOKEN = "<|endbayt|>"

def build_prompt(title: str, meter: str, theme: str = "عامة", qafiyah: str = "م") -> str:
    return f"""أنشئ قصيدة عربية فصيحة وفقاً للمواصفات التالية:

العنوان: {title}
البحر: {meter}
نوع القصيدة: {theme}
القافية: {qafiyah}

مواصفات القصيدة:
- التزام بالبحر {meter}
- كل بيت ينتهي بحرف :{qafiyah}
- متناسقة مع موضوع :{title}

الهيكل المطلوب:
{ST_BAYT_TOKEN} الشطر الأول {VERSE_TOKEN} الشطر الثاني {ED_BAYT_TOKEN}

اكتب القصيدة:"""


In [ ]:
# Define the input details
title = "جامعة سعود منبع الفخر والأمل"
meter = "الطويل"
theme = "قصيدة قصيره"
qafiyah = "ل"

# Build the prompt
prompt = build_prompt(title, meter, theme, qafiyah)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Tokenize and generate
inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
gen_ids = model.generate(
    **inputs,
    max_new_tokens=128,
    temperature=0.7,
    top_p=0.9,
    do_sample=True,
    pad_token_id=tokenizer.pad_token_id,
    eos_token_id=tokenizer.eos_token_id
)

# Decode the generated text
generated_text = tokenizer.decode(
    gen_ids[0][inputs.input_ids.shape[-1]:],
    skip_special_tokens=False
)

print("\nGenerated by Fine-tuned Model:")
print("="*80)
print(generated_text)
print("="*80)


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")



Generated by Fine-tuned Model:
[/يب :جامعة سعود منبع الفخر والأمل <|verse|> جامعة للعلماء من كل أهل <|endbayt|> جامعة للحكمة والصدق والنهى <|verse|> جامعة للعلم والمجد والفضل <|endbayt|> جامعة للسعي والعزم والعلا <|verse|> جامعة للعلم والعمل والعدل <|endbayt|> جامعة للحلم والنهى والبلاغة <|verse|> جامعة للعدل والشرف والمثل <|endbayt|> جامعة للشرف والصدق والوفا <|verse|> جامعة للحلم والعزم والعمل <|endbayt|>


In [ ]:
import openai

# Set your OpenAI API Key

# Build the enhancement prompt for GPT
gpt_prompt = f"""
أنت شاعر عربي فصيح، وخبير بالعروض والقوافي.

مهمتك:
- مراجعة القصيدة التالية لغويًا وعروضيًا.
- تحسين جودة الأبيات مع الحفاظ على البحر والقافية الأصلية.
- تأكد من أن القافية ثابتة في جميع الأبيات، ولا تتغير أثناء التحسين.
- حذف أي تكرار زائد دون الإخلال بالمعنى أو الوزن.
- الحفاظ على الفكرة الأساسية للقصيدة دون تغييرها.

ملاحظات مهمة:
- لا تضف شروحات أو تعليقات، فقط أعِد كتابة القصيدة بشكل محسن.
- احرص أن تكون الأبيات متوازنة عروضياً ومتسقة من حيث السجع والوزن.
- تأكد من أن القافية لا تتغير وأنها تبقى ثابتة في جميع الأبيات.

القصيدة:

\"\"\"
{generated_text}
\"\"\"
"""

# Request GPT to enhance
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "أنت شاعر محترف ومُدقق عروض."},
        {"role": "user", "content": gpt_prompt}
    ],
    temperature=0.4,
    max_tokens=800
)

# Get enhanced poem
enhanced_poem = response['choices'][0]['message']['content']

print("\nEnhanced by GPT:")
print("="*80)
print(enhanced_poem)
print("="*80)



Enhanced by GPT:
جامعة سعود، مصب الفخر والأمل
مهد العلماء من كل حي ودار
جامعة الحكمة، الصدق، والنهى
بيت العلم، المجد، والفضل البار
جامعة السعي، العزم، والعلا
مدرسة العلم، العمل، والعدل الجزار
جامعة الحلم، النهى، والبلاغة
محكمة العدل، الشرف، والمثل النضار
جامعة الشرف، الصدق، والوفا
معقل الحلم، العزم، والعمل الصرار
